First, we need to import related packages

In [101]:
# Import necessary libraries
from botbuilder.schema import Activity
from flask import Flask, request, jsonify
from botbuilder.core import BotFrameworkAdapterSettings, TurnContext
from teams import Application, ApplicationOptions, TurnState
import sys
import traceback
import os


> You can find the following code in `config.py`

Then we can initialize three constants. **Config** is from config.py, containing `port`, `app_id` and `app_password`.

In [102]:
# Create the bot application
class Config:
    """Bot Configuration"""
    port = 3978
    app_id = os.environ.get("BOT_ID", "")
    app_password = os.environ.get("BOT_PASSWORD", "")

Now we will define a bot **app** using a 'BotFrameworkAdapterSettings' with `config.app_id` and `config.app_password` as a `BotFrameworkAdapterSettings` auth setting.

In [ ]:

config = Config()
app = Application[TurnState](
    ApplicationOptions(
        bot_app_id=config.app_id,
        auth=BotFrameworkAdapterSettings(
            app_id=config.app_id,
            app_password=config.app_password,
        ),
    )
)

Then we can define `on_message()` message handler of **app** now.

In [58]:

# Define the message handler
@app.activity("message")
async def on_message(context: TurnContext, _state: TurnState):
    await context.send_activity(f"you said: {context.activity.text}")
    print(f"you said: {context.activity.text}")
    return True

@app.activity("no_mes")
async def on_no_message(context: TurnContext, _state: TurnState):
    await context.send_activity(f"you said: no message")
    print(f"you said: no message")
    return True

# Define the error handler
@app.error
async def on_error(context: TurnContext, error: Exception):
    # This check writes out errors to console log .vs. app insights.
    # NOTE: In production environment, you should consider logging this to Azure
    #       application insights.
    print(f"\n [on_turn_error] unhandled error: {error}", file=sys.stderr)
    traceback.print_exc()

    # Send a message to the user
    await context.send_activity("The bot encountered an error or bug.")


The provided test case is testing the `on_message()` function. The purpose of this test case is to verify the behavior of the `on_message()` function when it receives a user message. It ensures that the function correctly processes the message and sends a response activity with the appropriate content.

If the `on_message()` works correctly, it will prints the string `"you said: Hello, test message!"`.

In [ ]:
from botbuilder.core import TurnContext
from unittest.mock import MagicMock

async def test_on_message():
    # Mock the content of the context object
    context = MagicMock(spec=TurnContext)
    context.activity.channel_id = "mock_channel"
    context.activity.from_property.id = "mock_user"

    # Set the activity properties
    context.activity.text = "Hello, test messages!!"

    # Call the on_message() function
    await on_message(context, None)

# Call the test function
await test_on_message()

In [105]:
# Define the Flask API
api = Flask(__name__)
# Define the route for receiving messages
@api.route('/api/messages', methods=['POST'])
async def on_messages():
    print("in on_messages")
    activity = Activity().deserialize(request.json)

    auth_header = request.headers['Authorization'] if 'Authorization' in request.headers else ''
    response = await app.process_activity(activity, auth_header)
    
    if response:
        return jsonify(response.body), response.status
    return '', 200

In [ ]:

# Run the Flask API
if __name__ == "__main__":
    try:
        api.run(host='localhost', port=config.port)
    except Exception as error:
        raise error